# Data Cleaning for Resume Extraction Project - Sukriti Macker

In [9]:
# # Installing required libraries
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [13]:
#Importing Basic Libraries 
import numpy as np
import pandas as pd
import re
#Import NLP libraries
import string
import contractions
import spacy
import nltk
from nltk.corpus import stopwords
#program to display warning a message 
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reading the json file and storing it in dataframe 'df'
df = pd.read_json("/Users/sukritimacker/Downloads/resume_dataset/Resume.json", lines = True)

In [3]:
df.head(5)

,content,annotation
0,Govardhana K\nSenior Software Engineer\n\nBeng...,"[{'label': ['Companies worked at'], 'points': ..."
1,"Harini Komaravelli\nTest Analyst at Oracle, Hy...","[{'label': ['Companies worked at'], 'points': ..."
2,Hartej Kathuria\nData Analyst Intern - Oracle ...,"[{'label': ['Skills'], 'points': [{'start': 22..."
3,Ijas Nizamuddin\nAssociate Consultant - State ...,"[{'label': ['Skills'], 'points': [{'start': 46..."
4,"Imgeeyaul Ansari\njava developer\n\nPune, Maha...","[{'label': ['Skills'], 'points': [{'start': 18..."


In [4]:
## Understanding the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   content     200 non-null    object
 1   annotation  200 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [5]:
df['content'][0]

'Govardhana K\nSenior Software Engineer\n\nBengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/\nb2de315d95905b68\n\nTotal IT experience 5 Years 6 Months\nCloud Lending Solutions INC 4 Month • Salesforce Developer\nOracle 5 Years 2 Month • Core Java Developer\nLanguages Core Java, Go Lang\nOracle PL-SQL programming,\nSales Force Developer with APEX.\n\nDesignations & Promotions\n\nWilling to relocate: Anywhere\n\nWORK EXPERIENCE\n\nSenior Software Engineer\n\nCloud Lending Solutions -  Bangalore, Karnataka -\n\nJanuary 2018 to Present\n\nPresent\n\nSenior Consultant\n\nOracle -  Bangalore, Karnataka -\n\nNovember 2016 to December 2017\n\nStaff Consultant\n\nOracle -  Bangalore, Karnataka -\n\nJanuary 2014 to October 2016\n\nAssociate Consultant\n\nOracle -  Bangalore, Karnataka -\n\nNovember 2012 to December 2013\n\nEDUCATION\n\nB.E in Computer Science Engineering\n\nAdithya Institute of Technology -  Tamil Nadu\n\nSeptember 2008 to June 2012\n\nhttps://www.

In [6]:
df['annotation'][0]

[{'label': ['Companies worked at'],
  'points': [{'start': 1749, 'end': 1754, 'text': 'Oracle'}]},
 {'label': ['Companies worked at'],
  'points': [{'start': 1696, 'end': 1701, 'text': 'Oracle'}]},
 {'label': ['Companies worked at'],
  'points': [{'start': 1417, 'end': 1422, 'text': 'Oracle'}]},
 {'label': ['Skills'],
  'points': [{'start': 1356,
    'end': 1792,
    'text': 'Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle\nPL-SQL programming, Sales Force with APEX.\nTools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer,\nPL/SQL Developer, WinSCP, Putty\nWeb Technologies: JavaScript, XML, HTML, Webservice\n\nOperating Systems: Linux, Windows\nVersion control system SVN & Git-Hub\nDatabases: Oracle\nMiddleware: Web logic, OC4J\nProduct FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x'}]},
 {'label': ['Companies worked at'],
  'points': [{'start': 1209, 'end': 1214, 'text': 'Oracle'}]},
 {'label': ['Skills'],
  'points': [{'start': 1136,
    'end': 1247,


## Handling "Content"

In [23]:
## Function to clean the text.
def clean_text(text):
    # Lower the text
    cleaned_text = text.lower()
    
    #Remove contractions. For eg: "wouldn't" to "would not"
    cleaned_text = contractions.fix(text)
    
    # Remove newline characters
    cleaned_text = text.replace('\n', ' ')
    
    # Remove "http://" and "https://" prefixes from URLs
    cleaned_text = re.sub(r'https?://', '', cleaned_text)
    
    # Remove special characters and digits (except for '@' and '.')
    cleaned_text = re.sub(r'[^a-zA-Z0-9@\.\s]', '', cleaned_text)
    
    # Remove extra whitespaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

In [24]:
df['cleaned_content'] = df['content'].apply(clean_text)

In [25]:
df['cleaned_content'][0]

'Govardhana K Senior Software Engineer Bengaluru Karnataka Karnataka Email me on Indeed indeed.comrGovardhanaK b2de315d95905b68 Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month Salesforce Developer Oracle 5 Years 2 Month Core Java Developer Languages Core Java Go Lang Oracle PLSQL programming Sales Force Developer with APEX. Designations Promotions Willing to relocate Anywhere WORK EXPERIENCE Senior Software Engineer Cloud Lending Solutions Bangalore Karnataka January 2018 to Present Present Senior Consultant Oracle Bangalore Karnataka November 2016 to December 2017 Staff Consultant Oracle Bangalore Karnataka January 2014 to October 2016 Associate Consultant Oracle Bangalore Karnataka November 2012 to December 2013 EDUCATION B.E in Computer Science Engineering Adithya Institute of Technology Tamil Nadu September 2008 to June 2012 www.indeed.comrGovardhanaKb2de315d95905b68isidrexdownloadikwdownloadtopcoIN www.indeed.comrGovardhanaKb2de315d95905b68isidrexdownloadi

In [26]:
## Function to tokenize the text
def tokenization(text):
    # Load the English language model in spaCy
    nlp = spacy.load("en_core_web_sm")
    
    # Process the text using spaCy
    doc = nlp(text)
    
    # Extract the tokens from the processed document
    tokens = [token.text for token in doc]
    
    return tokens

In [34]:
df['tokenized_content'] = df['cleaned_content'].apply(tokenization)

In [35]:
df['tokenized_content']

0      [Govardhana, K, Senior, Software, Engineer, Be...
1      [Harini, Komaravelli, Test, Analyst, at, Oracl...
2      [Hartej, Kathuria, Data, Analyst, Intern, Orac...
3      [Ijas, Nizamuddin, Associate, Consultant, Stat...
4      [Imgeeyaul, Ansari, java, developer, Pune, Mah...
                             ...                        
195    [Mansi, Thanki, Student, Jamnagar, Gujarat, Em...
196    [Anil, Kumar, Microsoft, Azure, Basic, Managem...
197    [Siddharth, Choudhary, Microsoft, Office, Suit...
198    [Valarmathi, Dhandapani, Investment, Banking, ...
199    [Pradeep, Kumar, Security, Analyst, in, Infosy...
Name: tokenized_content, Length: 200, dtype: object

In [38]:
## Removing the stop words in the text
SetOfStopWords= set(stopwords.words('english')+['``',"''"])
totalWords= []

for requiredWords in df['tokenized_content']:
    for word in requiredWords:
        if word not in SetOfStopWords and word not in string.punctuation:

            totalWords.append(word)

In [39]:
totalWords

['Govardhana',
 'K',
 'Senior',
 'Software',
 'Engineer',
 'Bengaluru',
 'Karnataka',
 'Karnataka',
 'Email',
 'Indeed',
 'indeed.comrGovardhanaK',
 'b2de315d95905b68',
 'Total',
 'IT',
 'experience',
 '5',
 'Years',
 '6',
 'Months',
 'Cloud',
 'Lending',
 'Solutions',
 'INC',
 '4',
 'Month',
 'Salesforce',
 'Developer',
 'Oracle',
 '5',
 'Years',
 '2',
 'Month',
 'Core',
 'Java',
 'Developer',
 'Languages',
 'Core',
 'Java',
 'Go',
 'Lang',
 'Oracle',
 'PLSQL',
 'programming',
 'Sales',
 'Force',
 'Developer',
 'APEX',
 'Designations',
 'Promotions',
 'Willing',
 'relocate',
 'Anywhere',
 'WORK',
 'EXPERIENCE',
 'Senior',
 'Software',
 'Engineer',
 'Cloud',
 'Lending',
 'Solutions',
 'Bangalore',
 'Karnataka',
 'January',
 '2018',
 'Present',
 'Present',
 'Senior',
 'Consultant',
 'Oracle',
 'Bangalore',
 'Karnataka',
 'November',
 '2016',
 'December',
 '2017',
 'Staff',
 'Consultant',
 'Oracle',
 'Bangalore',
 'Karnataka',
 'January',
 '2014',
 'October',
 '2016',
 'Associate',
 'Con

In [40]:
'''
The FreqDist function gives the user the frequency distribution of all the words in the text. It helps us to get a
quick overview of the main topic or maybe the theme of the text.
'''
wordfreqdist = nltk.FreqDist(totalWords)

In [41]:
wordfreqdist

FreqDist({'Microsoft': 384, 'team': 371, 'using': 358, 'years': 336, '1': 325, 'Management': 324, 'year': 291, 'management': 288, 'Oracle': 281, 'Project': 246, ...})

In [43]:
mostcommon = wordfreqdist.most_common(20)
mostcommon

[('Microsoft', 384),
 ('team', 371),
 ('using', 358),
 ('years', 336),
 ('1', 325),
 ('Management', 324),
 ('year', 291),
 ('management', 288),
 ('Oracle', 281),
 ('Project', 246),
 ('SAP', 243),
 ('data', 226),
 ('project', 219),
 ('Email', 212),
 ('testing', 209),
 ('Test', 208),
 ('Infosys', 208),
 ('EXPERIENCE', 206),
 ('experience', 204),
 ('Testing', 202)]

## Handlling 'Annotation'

In [44]:
def extract_text_from_labels(data):
    label_text_dict = {}
    for item in data:
        if 'label' in item and isinstance(item['label'], list) and item['label']:
            label = item['label'][0]
            points = item.get('points', [])
            for point in points:
                if 'text' in point:
                    text = point['text']
                    if label == 'Companies worked at':
                        if label in label_text_dict:
                            label_text_dict[label].add(text)
                        else:
                            label_text_dict[label] = {text}
                    else:
                        if label in label_text_dict:
                            label_text_dict[label].append(text)
                        else:
                            label_text_dict[label] = [text]
    return label_text_dict

In [45]:
df['annotation_info_extraction'] = df['annotation'].apply(extract_text_from_labels) 

In [46]:
df['annotation_info_extraction'][0]

{'Companies worked at': {'Cloud Lending Solutions', 'Oracle'},
 'Skills': ['Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle\nPL-SQL programming, Sales Force with APEX.\nTools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer,\nPL/SQL Developer, WinSCP, Putty\nWeb Technologies: JavaScript, XML, HTML, Webservice\n\nOperating Systems: Linux, Windows\nVersion control system SVN & Git-Hub\nDatabases: Oracle\nMiddleware: Web logic, OC4J\nProduct FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x',
  'APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years),\nAlgorithms (3 years)\n'],
 'Graduation Year': ['2012', '2012'],
 'College Name': ['Adithya Institute of Technology'],
 'Degree': ['B.E in Computer Science Engineering'],
 'Designation': ['Associate Consultant',
  'Staff Consultant',
  'Senior Consultant\n',
  'Senior Software Engineer\n',
  'Senior Software Engineer\n'],
 'Email Address': ['indeed.com/r/Govardhana-K/\nb2de315d

In [47]:
def clean_annotation(dictionary):
    cleaned_dict = {}
    for key, value in dictionary.items():
        if isinstance(value, list):
            cleaned_dict[key] = [text.replace('\n', '') for text in value]
        elif isinstance(value, set):
            cleaned_dict[key] = {text.replace('\n', '') for text in value}
        else:
            cleaned_dict[key] = value.replace('\n', '')
    return cleaned_dict

In [48]:
df['annotation_info_extraction'] = df['annotation_info_extraction'].apply(clean_annotation)

In [49]:
df['annotation_info_extraction'][0]

{'Companies worked at': {'Cloud Lending Solutions', 'Oracle'},
 'Skills': ['Languages: Core Java, Go Lang, Data Structures & Algorithms, OraclePL-SQL programming, Sales Force with APEX.Tools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer,PL/SQL Developer, WinSCP, PuttyWeb Technologies: JavaScript, XML, HTML, WebserviceOperating Systems: Linux, WindowsVersion control system SVN & Git-HubDatabases: OracleMiddleware: Web logic, OC4JProduct FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x',
  'APEX. (Less than 1 year), Data Structures (3 years), FLEXCUBE (5 years), Oracle (5 years),Algorithms (3 years)'],
 'Graduation Year': ['2012', '2012'],
 'College Name': ['Adithya Institute of Technology'],
 'Degree': ['B.E in Computer Science Engineering'],
 'Designation': ['Associate Consultant',
  'Staff Consultant',
  'Senior Consultant',
  'Senior Software Engineer',
  'Senior Software Engineer'],
 'Email Address': ['indeed.com/r/Govardhana-K/b2de315d95905b68'],
 'Location': ['Benga

In [53]:
df

,content,annotation,cleaned_content,tokenized_content,annotation_info_extraction
0,Govardhana K\nSenior Software Engineer\n\nBeng...,"[{'label': ['Companies worked at'], 'points': ...",Govardhana K Senior Software Engineer Bengalur...,"[Govardhana, K, Senior, Software, Engineer, Be...","{'Companies worked at': {'Oracle', 'Cloud Lend..."
1,"Harini Komaravelli\nTest Analyst at Oracle, Hy...","[{'label': ['Companies worked at'], 'points': ...",Harini Komaravelli Test Analyst at Oracle Hyde...,"[Harini, Komaravelli, Test, Analyst, at, Oracl...","{'Companies worked at': {'Oracle'}, 'Skills': ..."
2,Hartej Kathuria\nData Analyst Intern - Oracle ...,"[{'label': ['Skills'], 'points': [{'start': 22...",Hartej Kathuria Data Analyst Intern Oracle Ret...,"[Hartej, Kathuria, Data, Analyst, Intern, Orac...",{'Skills': [' Languages & Technologies: Python...
3,Ijas Nizamuddin\nAssociate Consultant - State ...,"[{'label': ['Skills'], 'points': [{'start': 46...",Ijas Nizamuddin Associate Consultant State Str...,"[Ijas, Nizamuddin, Associate, Consultant, Stat...",{'Skills': ['Languages: Core JavaFront end/GUI...
4,"Imgeeyaul Ansari\njava developer\n\nPune, Maha...","[{'label': ['Skills'], 'points': [{'start': 18...",Imgeeyaul Ansari java developer Pune Maharasht...,"[Imgeeyaul, Ansari, java, developer, Pune, Mah...","{'Skills': ['rogramming Languages :C/C++, angu..."
...,...,...,...,...,...
195,"Mansi Thanki\nStudent\n\nJamnagar, Gujarat - E...","[{'label': ['College Name'], 'points': [{'star...",Mansi Thanki Student Jamnagar Gujarat Email me...,"[Mansi, Thanki, Student, Jamnagar, Gujarat, Em...",{'College Name': ['Government Engineering Coll...
196,Anil Kumar\nMicrosoft Azure (Basic Management)...,"[{'label': ['Location'], 'points': [{'start': ...",Anil Kumar Microsoft Azure Basic Management De...,"[Anil, Kumar, Microsoft, Azure, Basic, Managem...","{'Location': ['Delhi', 'Delhi', 'Delhi', 'Delh..."
197,Siddharth Choudhary\nMicrosoft Office Suite - ...,"[{'label': ['Skills'], 'points': [{'start': 78...",Siddharth Choudhary Microsoft Office Suite Exp...,"[Siddharth, Choudhary, Microsoft, Office, Suit...","{'Skills': ['AUDITING (Less than 1 year), CFA ..."
198,Valarmathi Dhandapani\nInvestment Banking Oper...,"[{'label': ['Skills'], 'points': [{'start': 92...",Valarmathi Dhandapani Investment Banking Opera...,"[Valarmathi, Dhandapani, Investment, Banking, ...","{'Skills': ['Excel (10+ years), Operations (7 ..."


In [56]:
df.to_csv("/Users/sukritimacker/Downloads/resume_dataset/cleaned_df.csv")